**Versions of tensorflow and keras that worked on my computer**

In [ ]:
!pip install tensorflow-gpu==2.4.0

In [ ]:
!pip install tensorflow==2.4.0

In [ ]:
!pip install keras==2.4.0

**Mount google drive and unzip photos**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! unzip "/content/gdrive/MyDrive/CS271Project/train-faces.zip"

**Imports**

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2

from imageio import imread
from skimage.transform import resize
from tqdm import tqdm

In [ ]:
!pip install keras_vggface

In [ ]:
!pip install keras_applications

In [ ]:
from tensorflow import keras 
from collections import defaultdict
from glob import glob
from random import choice, sample

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Input, Dense, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, Add
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

from keras_vggface import utils
from keras_vggface.vggface import VGGFace

In [ ]:
# make sure tensorflow is using GPUs

import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()  #disable for tensorFlow V2
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

Instructions for updating:
non-resource variables are not supported in the long term


**Baseline code, provided from Kaggle competition**

In [ ]:
# Organizing file paths to correctly label related faces

train_folders_path = "/content/train-faces/**/**/*.jpg"
train_file_path = "/content/gdrive/MyDrive/CS271Project/train-pairs.csv"
val_families = "F09"

all_images = glob(train_folders_path)

# Separating folder 9 as the validation set
train_images = [x for x in all_images if val_families not in x]
val_images = [x for x in all_images if val_families in x]

train_person_to_images_map = defaultdict(list)

# Each person has a unique number, most have multiple images
ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]

for x in train_images:
    train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

val_person_to_images_map = defaultdict(list)

for x in val_images:
    val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

# Reading the pairs of people who are related
relationships = pd.read_csv(train_file_path)
relationships = list(zip(relationships.p1.values, relationships.p2.values))
relationships = [x for x in relationships if x[0] in ppl and x[1] in ppl]

# Related images in training and validation sets
train = [x for x in relationships if val_families not in x[0]]
val = [x for x in relationships if val_families in x[0]]

In [ ]:
# Read, resize, reshape image

def read_img(path):
    img = cv2.imread(path)
    img = np.array(img).astype(np.float)
    img = cv2.resize(img,(224, 224))  
    img = np.reshape(img,[224,224,3])
    return utils.preprocess_input(img, version=2)

In [ ]:
# Generator for model.fit
# X1 and X2 are image outputs for person 1 and person 2
# label shows if X1 and X2 are related (1) or not (0), one label per pair
# Batch size dictates how many (X1, X2) pairs and number of labels

def gen(list_tuples, person_to_images_map, batch_size=16):

    # list of people
    ppl = list(person_to_images_map.keys())

    while True:

        batch_tuples = sample(list_tuples, batch_size // 2)
        labels = [1] * len(batch_tuples)

        while len(batch_tuples) < batch_size:
            p1 = choice(ppl)
            p2 = choice(ppl)

            if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
                batch_tuples.append((p1, p2))
                labels.append(0)

        for x in batch_tuples:
            if not len(person_to_images_map[x[0]]):
                print(x[0])

        X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
        X1 = np.array([read_img(x) for x in X1])
      
        X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
        X2 = np.array([read_img(x) for x in X2])

        labels = np.asarray(labels, dtype=np.float32)

        yield (X1, X2), labels

**Model 1/6**

In [ ]:
def baseline_model1():

    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    base_model = VGGFace(model='resnet50', include_top=False)

    for x in base_model.layers[:-3]:
        x.trainable = True

    x1 = base_model(input_1)
    x2 = base_model(input_2)

    x1 = Concatenate(axis=-1)([GlobalMaxPool2D()(x1), GlobalAvgPool2D()(x1)])
    x2 = Concatenate(axis=-1)([GlobalMaxPool2D()(x2), GlobalAvgPool2D()(x2)])

    x3 = Subtract()([x1, x2])
    x3 = Multiply()([x3, x3])

    x = Multiply()([x1, x2])

    x = Concatenate(axis=-1)([x, x3])

    x = Dense(512, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    x = Dropout(0.5)(x)
    out = Dense(1, activation="sigmoid")(x)

    model = Model([input_1, input_2], out)

    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))

    model.summary()

    return model

In [ ]:
file_path = "vggface1.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.1, patience=10, verbose=1)

early_stop = EarlyStopping(monitor='loss', patience=5)

callbacks_list = [checkpoint, reduce_on_plateau, early_stop]

model1 = baseline_model1()

94699520/94694792 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vggface_resnet50 (Functional)   (None, None, None, 2 23561152    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
global_max_pooling2d (GlobalM

In [ ]:
model1.fit(gen(train, train_person_to_images_map, batch_size=16), use_multiprocessing=True,
                    validation_data=gen(val, val_person_to_images_map, batch_size=16), epochs=50, verbose=2,
                    workers=4, callbacks=callbacks_list, steps_per_epoch=100, validation_steps=50)

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '



Epoch 00001: val_acc improved from -inf to 0.55500, saving model to vggface1.h5
100/100 - 61s - loss: 14.7809 - acc: 0.5656 - val_loss: 13.9879 - val_acc: 0.5550
Epoch 2/50

Epoch 00002: val_acc improved from 0.55500 to 0.59875, saving model to vggface1.h5
100/100 - 35s - loss: 13.1111 - acc: 0.5644 - val_loss: 12.1873 - val_acc: 0.5987
Epoch 3/50

Epoch 00003: val_acc improved from 0.59875 to 0.60750, saving model to vggface1.h5
100/100 - 34s - loss: 11.3539 - acc: 0.6162 - val_loss: 11.1812 - val_acc: 0.6075
Epoch 4/50

Epoch 00004: val_acc improved from 0.60750 to 0.63250, saving model to vggface1.h5
100/100 - 34s - loss: 10.7223 - acc: 0.6231 - val_loss: 11.2719 - val_acc: 0.6325
Epoch 5/50

Epoch 00005: val_acc did not improve from 0.63250
100/100 - 32s - loss: 10.3910 - acc: 0.6456 - val_loss: 10.5940 - val_acc: 0.5962
Epoch 6/50

Epoch 00006: val_acc improved from 0.63250 to 0.66250, saving model to vggface1.h5
100/100 - 36s - loss: 10.2746 - acc: 0.6363 - val_loss: 10.2242 - v

**Model 2/6**

In [ ]:
def baseline_model2():

    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    base_model = VGGFace(model='resnet50', include_top=False)

    for x in base_model.layers[:-3]:
        x.trainable = True

    x1 = base_model(input_1)
    x2 = base_model(input_2)

    x1 = Concatenate(axis=-1)([GlobalMaxPool2D()(x1), GlobalAvgPool2D()(x1)])
    x2 = Concatenate(axis=-1)([GlobalMaxPool2D()(x2), GlobalAvgPool2D()(x2)])

    x3 = Subtract()([x1, x2])
    x3 = Multiply()([x3, x3])

    x1_2 = Multiply()([x1, x1])
    x2_2 = Multiply()([x2, x2])

    x = Subtract()([x1_2, x2_2])

    x = Concatenate(axis=-1)([x, x3])

    x = Dense(512, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    x = Dropout(0.5)(x)
    out = Dense(1, activation="sigmoid")(x)

    model = Model([input_1, input_2], out)

    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))

    model.summary()

    return model

In [ ]:
file_path = "vggface2.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.1, patience=10, verbose=1)

early_stop = EarlyStopping(monitor='loss', patience=5)

callbacks_list = [checkpoint, reduce_on_plateau, early_stop]

model2 = baseline_model2()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vggface_resnet50 (Functional)   (None, None, None, 2 23561152    input_4[0][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
global_max_pooling2d_2 (GlobalM (None, 2048)         0           vggface_resnet50[0][0]     

In [ ]:
model2.fit(gen(train, train_person_to_images_map, batch_size=16), use_multiprocessing=True,
                    validation_data=gen(val, val_person_to_images_map, batch_size=16), epochs=50, verbose=2,
                    workers=4, callbacks=callbacks_list, steps_per_epoch=100, validation_steps=50)

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '



Epoch 00001: val_acc improved from -inf to 0.60000, saving model to vggface2.h5
100/100 - 54s - loss: 17.0606 - acc: 0.5456 - val_loss: 15.8800 - val_acc: 0.6000
Epoch 2/50

Epoch 00002: val_acc improved from 0.60000 to 0.62375, saving model to vggface2.h5
100/100 - 34s - loss: 14.8142 - acc: 0.5763 - val_loss: 13.1450 - val_acc: 0.6237
Epoch 3/50

Epoch 00003: val_acc did not improve from 0.62375
100/100 - 32s - loss: 12.7821 - acc: 0.5962 - val_loss: 12.2247 - val_acc: 0.5938
Epoch 4/50

Epoch 00004: val_acc did not improve from 0.62375
100/100 - 33s - loss: 11.7104 - acc: 0.5931 - val_loss: 11.5239 - val_acc: 0.6150
Epoch 5/50

Epoch 00005: val_acc improved from 0.62375 to 0.62750, saving model to vggface2.h5
100/100 - 36s - loss: 11.1120 - acc: 0.5900 - val_loss: 11.2033 - val_acc: 0.6275
Epoch 6/50

Epoch 00006: val_acc did not improve from 0.62750
100/100 - 32s - loss: 10.6265 - acc: 0.6169 - val_loss: 10.9752 - val_acc: 0.6150
Epoch 7/50

Epoch 00007: val_acc did not improve fr

**Model 3/6**

In [ ]:
def baseline_model3():

    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    base_model = VGGFace(model='resnet50', include_top=False)

    for x in base_model.layers[:-3]:
        x.trainable = True

    x1 = base_model(input_1)
    x2 = base_model(input_2)

    x1 = Concatenate(axis=-1)([GlobalMaxPool2D()(x1), GlobalAvgPool2D()(x1)])
    x2 = Concatenate(axis=-1)([GlobalMaxPool2D()(x2), GlobalAvgPool2D()(x2)])

    x3 = Subtract()([x1, x2])
    x3 = Multiply()([x3, x3])

    x = Add()([x1, x2])

    x = Concatenate(axis=-1)([x, x3])

    x = Dense(512, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    x = Dropout(0.5)(x)
    out = Dense(1, activation="sigmoid")(x)

    model = Model([input_1, input_2], out)

    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))

    model.summary()

    return model

In [ ]:
file_path = "vggface3.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.1, patience=10, verbose=1)

early_stop = EarlyStopping(monitor='loss', patience=5)

callbacks_list = [checkpoint, reduce_on_plateau, early_stop]

model3 = baseline_model3()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vggface_resnet50 (Functional)   (None, None, None, 2 23561152    input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
global_max_pooling2d_4 (GlobalM (None, 2048)         0           vggface_resnet50[0][0]     

In [ ]:
model3.fit(gen(train, train_person_to_images_map, batch_size=16), use_multiprocessing=True,
                    validation_data=gen(val, val_person_to_images_map, batch_size=16), epochs=50, verbose=2,
                    workers=4, callbacks=callbacks_list, steps_per_epoch=100, validation_steps=50)

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '



Epoch 00001: val_acc improved from -inf to 0.55000, saving model to vggface3.h5
100/100 - 58s - loss: 14.7935 - acc: 0.5350 - val_loss: 13.6909 - val_acc: 0.5500
Epoch 2/50

Epoch 00002: val_acc improved from 0.55000 to 0.58000, saving model to vggface3.h5
100/100 - 35s - loss: 12.5616 - acc: 0.5825 - val_loss: 12.5986 - val_acc: 0.5800
Epoch 3/50

Epoch 00003: val_acc did not improve from 0.58000
100/100 - 33s - loss: 11.7204 - acc: 0.5938 - val_loss: 12.3995 - val_acc: 0.5362
Epoch 4/50

Epoch 00004: val_acc improved from 0.58000 to 0.61000, saving model to vggface3.h5
100/100 - 36s - loss: 11.0119 - acc: 0.6100 - val_loss: 10.6622 - val_acc: 0.6100
Epoch 5/50

Epoch 00005: val_acc improved from 0.61000 to 0.64250, saving model to vggface3.h5
100/100 - 35s - loss: 10.5105 - acc: 0.6175 - val_loss: 10.5323 - val_acc: 0.6425
Epoch 6/50

Epoch 00006: val_acc did not improve from 0.64250
100/100 - 33s - loss: 10.2999 - acc: 0.6288 - val_loss: 10.3969 - val_acc: 0.6250
Epoch 7/50

Epoch 

**Model 4/6**

In [ ]:
def baseline_model4():

    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    base_model = VGGFace(model='resnet50', include_top=False)

    for x in base_model.layers[:-3]:
        x.trainable = True

    x1 = base_model(input_1)
    x2 = base_model(input_2)

    x1 = Concatenate(axis=-1)([GlobalMaxPool2D()(x1), GlobalAvgPool2D()(x1)])
    x2 = Concatenate(axis=-1)([GlobalMaxPool2D()(x2), GlobalAvgPool2D()(x2)])

    x1_2 = Multiply()([x1, x1])
    x2_2 = Multiply()([x2, x2])

    x3 = Subtract()([x1_2, x2_2])

    x = Multiply()([x1, x2])

    x = Concatenate(axis=-1)([x, x3])

    x = Dense(512, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    x = Dropout(0.5)(x)
    out = Dense(1, activation="sigmoid")(x)

    model = Model([input_1, input_2], out)

    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))

    model.summary()

    return model

In [ ]:
file_path = "vggface4.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.1, patience=10, verbose=1)

early_stop = EarlyStopping(monitor='loss', patience=5)

callbacks_list = [checkpoint, reduce_on_plateau, early_stop]

model4 = baseline_model4()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vggface_resnet50 (Functional)   (None, None, None, 2 23561152    input_10[0][0]                   
                                                                 input_11[0][0]                   
__________________________________________________________________________________________________
global_max_pooling2d_6 (GlobalM (None, 2048)         0           vggface_resnet50[0][0]     

In [ ]:
model4.fit(gen(train, train_person_to_images_map, batch_size=16), use_multiprocessing=True,
                    validation_data=gen(val, val_person_to_images_map, batch_size=16), epochs=50, verbose=2,
                    workers=4, callbacks=callbacks_list, steps_per_epoch=100, validation_steps=50)

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '



Epoch 00001: val_acc improved from -inf to 0.56250, saving model to vggface4.h5
100/100 - 63s - loss: 16.3986 - acc: 0.5069 - val_loss: 15.1887 - val_acc: 0.5625
Epoch 2/50

Epoch 00002: val_acc improved from 0.56250 to 0.57875, saving model to vggface4.h5
100/100 - 36s - loss: 13.7753 - acc: 0.5694 - val_loss: 12.7614 - val_acc: 0.5788
Epoch 3/50

Epoch 00003: val_acc improved from 0.57875 to 0.61375, saving model to vggface4.h5
100/100 - 36s - loss: 11.9726 - acc: 0.5987 - val_loss: 11.5657 - val_acc: 0.6137
Epoch 4/50

Epoch 00004: val_acc did not improve from 0.61375
100/100 - 33s - loss: 10.9366 - acc: 0.6081 - val_loss: 10.8086 - val_acc: 0.6112
Epoch 5/50

Epoch 00005: val_acc improved from 0.61375 to 0.62625, saving model to vggface4.h5
100/100 - 36s - loss: 10.6625 - acc: 0.5956 - val_loss: 10.4303 - val_acc: 0.6263
Epoch 6/50

Epoch 00006: val_acc improved from 0.62625 to 0.65000, saving model to vggface4.h5
100/100 - 35s - loss: 10.4009 - acc: 0.6350 - val_loss: 10.4274 - v

**Model 5/6**

In [ ]:
def baseline_model5():

    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    base_model = VGGFace(model='resnet50', include_top=False)

    for x in base_model.layers[:-3]:
        x.trainable = True

    x1 = base_model(input_1)
    x2 = base_model(input_2)

    x1 = Concatenate(axis=-1)([GlobalMaxPool2D()(x1), GlobalAvgPool2D()(x1)])
    x2 = Concatenate(axis=-1)([GlobalMaxPool2D()(x2), GlobalAvgPool2D()(x2)])

    x1_2 = Multiply()([x1, x1])
    x2_2 = Multiply()([x2, x2])

    x3 = Subtract()([x1_2, x2_2])

    x = Add()([x1, x2])

    x = Concatenate(axis=-1)([x, x3])

    x = Dense(512, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    x = Dropout(0.5)(x)
    out = Dense(1, activation="sigmoid")(x)

    model = Model([input_1, input_2], out)

    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))

    model.summary()

    return model

In [ ]:
file_path = "vggface5.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.1, patience=10, verbose=1)

early_stop = EarlyStopping(monitor='loss', patience=5)

callbacks_list = [checkpoint, reduce_on_plateau, early_stop]

model5 = baseline_model5()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vggface_resnet50 (Functional)   (None, None, None, 2 23561152    input_13[0][0]                   
                                                                 input_14[0][0]                   
__________________________________________________________________________________________________
global_max_pooling2d_8 (GlobalM (None, 2048)         0           vggface_resnet50[0][0]     

In [ ]:
model5.fit(gen(train, train_person_to_images_map, batch_size=16), use_multiprocessing=True,
                    validation_data=gen(val, val_person_to_images_map, batch_size=16), epochs=50, verbose=2,
                    workers=4, callbacks=callbacks_list, steps_per_epoch=100, validation_steps=50)

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '



Epoch 00001: val_acc improved from -inf to 0.52375, saving model to vggface5.h5
100/100 - 67s - loss: 15.7264 - acc: 0.5113 - val_loss: 14.0345 - val_acc: 0.5238
Epoch 2/50

Epoch 00002: val_acc improved from 0.52375 to 0.57875, saving model to vggface5.h5
100/100 - 36s - loss: 13.4476 - acc: 0.5650 - val_loss: 12.6244 - val_acc: 0.5788
Epoch 3/50

Epoch 00003: val_acc improved from 0.57875 to 0.62250, saving model to vggface5.h5
100/100 - 36s - loss: 11.9635 - acc: 0.5619 - val_loss: 11.1103 - val_acc: 0.6225
Epoch 4/50

Epoch 00004: val_acc did not improve from 0.62250
100/100 - 34s - loss: 11.0286 - acc: 0.5981 - val_loss: 10.6914 - val_acc: 0.5962
Epoch 5/50

Epoch 00005: val_acc did not improve from 0.62250
100/100 - 36s - loss: 10.5653 - acc: 0.5900 - val_loss: 10.4195 - val_acc: 0.6087
Epoch 6/50

Epoch 00006: val_acc improved from 0.62250 to 0.64250, saving model to vggface5.h5
100/100 - 38s - loss: 10.2381 - acc: 0.6325 - val_loss: 10.2028 - val_acc: 0.6425
Epoch 7/50

Epoch 

**Model 6/6**

In [ ]:
def baseline_model6():

    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    base_model = VGGFace(model='resnet50', include_top=False)

    for x in base_model.layers[:-3]:
        x.trainable = True

    x1 = base_model(input_1)
    x2 = base_model(input_2)

    x1 = Concatenate(axis=-1)([GlobalMaxPool2D()(x1), GlobalAvgPool2D()(x1)])
    x2 = Concatenate(axis=-1)([GlobalMaxPool2D()(x2), GlobalAvgPool2D()(x2)])

    x3 = Add()([x1, x2])

    x = Multiply()([x1, x2])

    x = Concatenate(axis=-1)([x, x3])

    x = Dense(512, activation="relu",kernel_regularizer=regularizers.l2(0.01))(x)
    x = Dropout(0.5)(x)
    out = Dense(1, activation="sigmoid")(x)

    model = Model([input_1, input_2], out)

    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))

    model.summary()

    return model

In [ ]:
file_path = "vggface6.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.1, patience=10, verbose=1)

early_stop = EarlyStopping(monitor='loss', patience=5)

callbacks_list = [checkpoint, reduce_on_plateau, early_stop]

model6 = baseline_model6()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_17 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vggface_resnet50 (Functional)   (None, None, None, 2 23561152    input_16[0][0]                   
                                                                 input_17[0][0]                   
__________________________________________________________________________________________________
global_max_pooling2d_10 (Global (None, 2048)         0           vggface_resnet50[0][0]     

In [ ]:
model6.fit(gen(train, train_person_to_images_map, batch_size=16), use_multiprocessing=True,
                    validation_data=gen(val, val_person_to_images_map, batch_size=16), epochs=50, verbose=2,
                    workers=4, callbacks=callbacks_list, steps_per_epoch=100, validation_steps=50)

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '



Epoch 00001: val_acc improved from -inf to 0.60000, saving model to vggface6.h5
100/100 - 71s - loss: 11.5168 - acc: 0.5444 - val_loss: 10.9000 - val_acc: 0.6000
Epoch 2/50

Epoch 00002: val_acc improved from 0.60000 to 0.66000, saving model to vggface6.h5
100/100 - 37s - loss: 11.0135 - acc: 0.5756 - val_loss: 10.3701 - val_acc: 0.6600
Epoch 3/50

Epoch 00003: val_acc improved from 0.66000 to 0.66500, saving model to vggface6.h5
100/100 - 36s - loss: 10.4383 - acc: 0.6400 - val_loss: 10.2073 - val_acc: 0.6650
Epoch 4/50

Epoch 00004: val_acc did not improve from 0.66500
100/100 - 34s - loss: 10.2497 - acc: 0.6325 - val_loss: 10.2646 - val_acc: 0.6250
Epoch 5/50

Epoch 00005: val_acc did not improve from 0.66500
100/100 - 34s - loss: 10.1435 - acc: 0.6225 - val_loss: 10.0224 - val_acc: 0.6513
Epoch 6/50

Epoch 00006: val_acc did not improve from 0.66500
100/100 - 35s - loss: 10.0252 - acc: 0.6319 - val_loss: 9.9041 - val_acc: 0.6625
Epoch 7/50

Epoch 00007: val_acc improved from 0.665